In [306]:
#경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

#데이터 불러오기
import pandas as pd

In [307]:
poketmon_df = pd.read_csv('datasets_poketmon.csv', encoding='latin1')

In [308]:
#데이터 확인
poketmon_df.head()

,#,Name,HP,Attack,Defense,Speed,Generation,Legendary
0,1,Bulbasaur,45,49,49,45,1,False
1,2,Ivysaur,60,62,63,60,1,False
2,3,Venusaur,80,82,83,80,1,False
3,3,VenusaurMega Venusaur,80,100,123,80,1,False
4,4,Charmander,39,52,43,65,1,False


In [309]:
#데이터 통계
poketmon_df.describe()

#통계를 보았을 때 50% (중앙값)와 mean 값이 비슷한 경우 좋은 데이터

,#,HP,Attack,Defense,Speed,Generation
count,799.000000,799.000000,799.000000,799.000000,799.000000,799.000000
mean,363.227785,69.287860,79.028786,73.884856,68.300375,3.326658
std,208.144712,25.537377,32.468345,31.179997,29.071468,1.660292
min,1.000000,1.000000,5.000000,5.000000,5.000000,1.000000
25%,185.500000,50.000000,55.000000,50.000000,45.000000,2.000000
50%,365.000000,65.000000,75.000000,70.000000,65.000000,3.000000
75%,539.500000,80.000000,100.000000,90.000000,90.000000,5.000000
max,721.000000,255.000000,190.000000,230.000000,180.000000,6.000000


In [310]:
#전설 포켓몬 일반 포켓몬 개수 확인
poketmon_df['Legendary'].value_counts()

False    734
True      65
Name: Legendary, dtype: int64

In [311]:
#불필요한 컬럼 제거
poketmon_df.drop(columns=['#', 'Name'], inplace=True)

In [312]:
#데이터와 정답값 분리
data = poketmon_df.drop(columns=['Legendary'])
label = poketmon_df['Legendary']

In [313]:
#훈련 데이터와 테스트 데이터 나누기
from sklearn.model_selection import train_test_split

In [314]:
#무작위 시드값
random_state = 35

x_train, x_test, y_train, y_test = train_test_split(
    data, label, test_size=0.2, random_state=random_state
)

In [315]:
#머신러닝 모델 학습
from sklearn.tree import DecisionTreeClassifier #결정트리
from sklearn.linear_model import LogisticRegression #로지스틱회귀
from sklearn.ensemble import RandomForestClassifier #랜덤포레스트

In [316]:
dtc = DecisionTreeClassifier()
dtc.fit(x_train, y_train)
pred_dtc = dtc.predict(x_test)

In [317]:
lr = LogisticRegression()
lr.fit(x_train, y_train)
pred_lr = lr.predict(x_test)

In [318]:
rfc = RandomForestClassifier()
rfc.fit(x_train, y_train)
pred_rfc = rfc.predict(x_test)

In [319]:
#학습 성능평가
from sklearn.metrics import classification_report

In [320]:
#Logistic Regression 평가
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

       False       0.97      1.00      0.99       148
        True       1.00      0.67      0.80        12

    accuracy                           0.97       160
   macro avg       0.99      0.83      0.89       160
weighted avg       0.98      0.97      0.97       160



In [321]:
#Decision Tree 평가
print(classification_report(y_test, pred_dtc))

              precision    recall  f1-score   support

       False       0.99      0.96      0.97       148
        True       0.62      0.83      0.71        12

    accuracy                           0.95       160
   macro avg       0.81      0.90      0.84       160
weighted avg       0.96      0.95      0.95       160



In [322]:
#Random Forest 평가
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

       False       0.97      1.00      0.99       148
        True       1.00      0.67      0.80        12

    accuracy                           0.97       160
   macro avg       0.99      0.83      0.89       160
weighted avg       0.98      0.97      0.97       160



In [323]:
#데이터 정규화
from sklearn.preprocessing import MinMaxScaler

In [324]:
#수치형 데이터
num_data = data[['HP', 'Attack', 'Defense', 'Speed']]

#범주형 데이터
cat_data = data[['Generation']]

In [325]:
#수치형 데이터 정규화
scaler = MinMaxScaler()
num_data_scaled = scaler.fit_transform(num_data) #결과값은 넘파이 배열 형태

#데이터 프레임을 합치기 위해 넘파이 배열을 데이터프레임으로 만들기
print(num_data.columns)
num_data_scaled = pd.DataFrame(data=num_data_scaled, columns=num_data.columns) # data 는 데이터프레임에 들어갈 데이터, columns 는 데이터프레임에 지정할 컬럼 이름

#수치형 데이터와 범주형 데이터 합치기
data_minmax = pd.concat([num_data_scaled, cat_data], axis=1)
data_minmax.head()

Index(['HP', 'Attack', 'Defense', 'Speed'], dtype='object')


,HP,Attack,Defense,Speed,Generation
0,0.173228,0.237838,0.195556,0.228571,1
1,0.232283,0.308108,0.257778,0.314286,1
2,0.311024,0.416216,0.346667,0.428571,1
3,0.311024,0.513514,0.524444,0.428571,1
4,0.149606,0.254054,0.168889,0.342857,1


In [326]:
#정규화된 데이터 나누기
x_train, x_test, y_train, y_test = train_test_split(
    data_minmax, label, test_size=0.2, random_state=random_state
)

In [327]:
#모델 학습
lr.fit(x_train, y_train)
pred_lr = lr.predict(x_test)

dtc.fit(x_train, y_train)
pred_dtc = dtc.predict(x_test)

rfc.fit(x_train, y_train)
pred_rfc = rfc.predict(x_test)

In [328]:
#모델 학습 평가
print(classification_report(y_test, pred_lr)) #Logistic Regression

              precision    recall  f1-score   support

       False       0.94      1.00      0.97       148
        True       1.00      0.17      0.29        12

    accuracy                           0.94       160
   macro avg       0.97      0.58      0.63       160
weighted avg       0.94      0.94      0.92       160



In [329]:
print(classification_report(y_test, pred_dtc)) #Decision Tree

              precision    recall  f1-score   support

       False       0.99      0.96      0.98       148
        True       0.65      0.92      0.76        12

    accuracy                           0.96       160
   macro avg       0.82      0.94      0.87       160
weighted avg       0.97      0.96      0.96       160



In [330]:
print(classification_report(y_test, pred_rfc)) #Random Forest

              precision    recall  f1-score   support

       False       0.97      1.00      0.99       148
        True       1.00      0.67      0.80        12

    accuracy                           0.97       160
   macro avg       0.99      0.83      0.89       160
weighted avg       0.98      0.97      0.97       160



In [331]:
#데이터 표준화
from sklearn.preprocessing import StandardScaler

In [332]:
scaler = StandardScaler()
num_data_scaled = scaler.fit_transform(num_data) #결과값은 넘파이 배열 형태

#데이터 프레임을 합치기 위해 넘파이 배열을 데이터프레임으로 만들기
print(num_data.columns)
num_data_scaled = pd.DataFrame(data=num_data_scaled, columns=num_data.columns) # data 는 데이터프레임에 들어갈 데이터, columns 는 데이터프레임에 지정할 컬럼 이름

#수치형 데이터와 범주형 데이터 합치기
data_standard = pd.concat([num_data_scaled, cat_data], axis=1)
data_standard.head()

Index(['HP', 'Attack', 'Defense', 'Speed'], dtype='object')


,HP,Attack,Defense,Speed,Generation
0,-0.951667,-0.925443,-0.798603,-0.801988,1
1,-0.363925,-0.524802,-0.349316,-0.285695,1
2,0.419732,0.091568,0.292523,0.402696,1
3,0.419732,0.646302,1.576200,0.402696,1
4,-1.186764,-0.832987,-0.991155,-0.113597,1


In [333]:
x_train, x_test, y_train, y_test = train_test_split(
    data_standard, label, test_size=0.2, random_state=random_state
)

In [334]:
#모델 학습
lr.fit(x_train, y_train)
pred_lr = lr.predict(x_test)

dtc.fit(x_train, y_train)
pred_dtc = dtc.predict(x_test)

rfc.fit(x_train, y_train)
pred_rfc = rfc.predict(x_test)

In [335]:
#모델 학습 평가
print(classification_report(y_test, pred_lr)) #Logistic Regression

              precision    recall  f1-score   support

       False       0.97      1.00      0.99       148
        True       1.00      0.67      0.80        12

    accuracy                           0.97       160
   macro avg       0.99      0.83      0.89       160
weighted avg       0.98      0.97      0.97       160



In [336]:
print(classification_report(y_test, pred_dtc)) #Decision Tree

              precision    recall  f1-score   support

       False       0.99      0.97      0.98       148
        True       0.67      0.83      0.74        12

    accuracy                           0.96       160
   macro avg       0.83      0.90      0.86       160
weighted avg       0.96      0.96      0.96       160



In [337]:
print(classification_report(y_test, pred_rfc)) #Random Forest

              precision    recall  f1-score   support

       False       0.97      1.00      0.99       148
        True       1.00      0.67      0.80        12

    accuracy                           0.97       160
   macro avg       0.99      0.83      0.89       160
weighted avg       0.98      0.97      0.97       160

